# GRU

In [6]:
import pandas as pd
import numpy as np
import copy
import re
import torch
from sklearn.model_selection import train_test_split
import random

In [7]:
import pandas as pd
test = pd.read_csv("amazon_reviews_us_Kitchen_v1_00.tsv", sep = '\t',error_bad_lines = False)
test['label'] = -1
a1 = test.loc[test['star_rating']==1].sample(50000)
a2 = test.loc[test['star_rating']==2].sample(50000)
a3 = test.loc[test['star_rating']==3].sample(50000)
a4 = test.loc[test['star_rating']==4].sample(50000)
a5 = test.loc[test['star_rating']==5].sample(50000)

b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 15 fields, saw 22\nSkipping line 246351: expected 15 fields, saw 22\nSkipping line 252364: expected 15 fields, saw 22\n'
b'Skipping line 267003: expected 15 fields, saw 22\nSkipping line 268957: expected 15 fields, saw 22\nSkipping line 303336: expected 15 field

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

In [8]:
new_test = pd.concat([a1,a2,a3,a4,a5])

In [9]:
new_test['lable'] = 1

In [10]:
new_test.label[test.star_rating>3] = 1
new_test.label[test.star_rating<3] = 2
new_test.label[test.star_rating==3] = 3
new_test = new_test[['label','review_body']] 

In [11]:
import gensim
model_1 = gensim.models.Word2Vec.load('save')

In [12]:
#data clearing
new_test['review_body'] = new_test['review_body'].str.lower()

def tag(x):
    return re.sub('<.*?>','',str(x))
new_test['review_body'] = new_test['review_body'].apply(lambda x:tag(x))

def url(x):
    return re.sub('(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]','',str(x))

new_test['review_body'] = new_test['review_body'].apply(lambda x:url(x))

import contractions
new_test['review_body'] = new_test['review_body'].apply(lambda x:contractions.fix(x))

def non_alphabetical(x):
    return re.sub('[^a-zA-Z\s]','',str(x))
new_test['review_body'] = new_test['review_body'].apply(lambda x:non_alphabetical(x))
def extra_space(x):
    return re.sub( ' +',' ',str(x))
new_test['review_body'] = new_test['review_body'].apply(lambda x:extra_space(x))

In [13]:
#Pre-processing

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = set(stopwords.words('english'))
from nltk import word_tokenize, pos_tag

def stop_words(x):
    word_tokens = word_tokenize(x)
    temp = []
    for i in word_tokens:
        if i not in stop_words_set:
            temp.append(i)
    return temp
new_test['review_body'] = new_test['review_body'].apply(lambda x:stop_words(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
new_test['y'] = -1

In [15]:
new_test['y'][new_test['label'] == 1] =0
new_test['y'][new_test['label'] == 2] =1
new_test['y'][new_test['label'] == 3] =2

In [16]:
binary = new_test.loc[new_test['y']!=2]

In [17]:
def mymodel_reviewtensor(x):
    max_review = 50
    review = torch.zeros(max_review,1,300)
    pad = torch.zeros(300)
    index = 0
    for i in x:
        if i in model_1.wv and index < 50:
            vector = model_1.wv [i]
            review[index][0] = torch.from_numpy(vector)
            index += 1
        else:
            continue
    if index < 50:
        review[index][0] = pad
    review = torch.nan_to_num(review)
    return review 

In [21]:
import gensim
model_1 = gensim.models.Word2Vec.load('save')

In [22]:
import gensim.downloader as api

opm = api.load("word2vec-google-news-300")

In [18]:
def opm_reviewtensor(x):
    max_review = 50
    ### create a matrix 
    review = torch.zeros(max_review,1,300)
    pad = torch.zeros(300)
    index = 0
    for i in x:
        if i in opm and index < 50:
            vector = opm [i]
            review[index][0] = torch.from_numpy(vector)
            index += 1
        else:
            continue
    if index < 50:
        review[index][0] = pad
    review = torch.nan_to_num(review)
    return review 
        

In [19]:
def data_vector(dataset,opm):
    res = []
    if opm == True:
        for x,y in dataset:
            x_vector = opm_reviewtensor(x)
            res.append((x_vector,y))
    if opm == False:
        for x,y in dataset:
            x_vector = mymodel_reviewtensor(x)
            res.append((x_vector,y))
    return res
            

In [23]:
binary

,label,review_body,y
3840037,2,"[one, highly, disappointed, product, years, wa...",1
3791453,2,"[bought, month, ago, bbb, times, use, make, si...",1
2539564,2,"[bought, brewer, jan, within, weeks, carafe, e...",1
3239783,2,"[bought, couple, clean, bottom, beta, fish, bo...",1
2740932,2,"[love, productbut, old, one, broke, wanted, ge...",1
...,...,...,...
2183853,1,"[blast, family, making, taking, pics]",0
1020136,1,"[great, purchase, please, came, time, loved]",0
1758315,1,"[perfect, glasses, love]",0
4175724,1,"[pans, first, quality, mediumheavy, gauge, sta...",0


## binary GRU my model

In [26]:
binary_data = list(zip(binary.review_body.values,binary.y.values))

In [29]:
train_set, test_set = train_test_split(binary_data, random_state = 19, test_size = 0.2)

In [30]:
test = data_vector(test_set, opm=False )

train = data_vector(train_set, opm=False)


In [36]:
import torch.nn as nn

In [34]:
# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
#valid_size = 0.2
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=True, num_workers=0)

test_loader=torch.utils.data.DataLoader(test, batch_size=batch_size, drop_last=True,num_workers=0)

In [49]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

GRU_model = GRU(300, 50, 2, 2).to(device)


In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(GRU_model.parameters(), lr=0.002)  

In [51]:
# Train the model
num_epochs = 1
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (x_i,y_i) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        x_i = x_i.reshape(-1, 50, 300).to(device)
        y_i = y_i.to(device)
        
        # Forward pass
        outputs = GRU_model(x_i)
        loss = criterion(outputs, y_i)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/1], Step [100/1600], Loss: 0.5400
Epoch [1/1], Step [200/1600], Loss: 0.4260
Epoch [1/1], Step [300/1600], Loss: 0.2985
Epoch [1/1], Step [400/1600], Loss: 0.2918
Epoch [1/1], Step [500/1600], Loss: 0.3736
Epoch [1/1], Step [600/1600], Loss: 0.3736
Epoch [1/1], Step [700/1600], Loss: 0.2612
Epoch [1/1], Step [800/1600], Loss: 0.3358
Epoch [1/1], Step [900/1600], Loss: 0.2936
Epoch [1/1], Step [1000/1600], Loss: 0.3211
Epoch [1/1], Step [1100/1600], Loss: 0.3775
Epoch [1/1], Step [1200/1600], Loss: 0.3397
Epoch [1/1], Step [1300/1600], Loss: 0.4303
Epoch [1/1], Step [1400/1600], Loss: 0.2620
Epoch [1/1], Step [1500/1600], Loss: 0.3403
Epoch [1/1], Step [1600/1600], Loss: 0.2838


In [54]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 50, 300).to(device)
        labels = labels.to(device)
        outputs = GRU_model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of binary GRU my model: {acc} %')

Accuracy of the network on the 10000 test images: 87.6475 %


In [55]:
binary_GRU_my_model = acc

In [56]:
print(f'Accuracy of binary GRU my model: {acc} %')

Accuracy of binary GRU my model: 87.6475 %


# binary GRU googel model

In [57]:
test = data_vector(test_set, opm=True )

train = data_vector(train_set, opm=True)

In [58]:
# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
#valid_size = 0.2
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=True, num_workers=0)

test_loader=torch.utils.data.DataLoader(test, batch_size=batch_size, drop_last=True,num_workers=0)

In [59]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

GRU_model = GRU(300, 50, 2, 2).to(device)

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(GRU_model.parameters(), lr=0.002)  

In [61]:
# Train the model
num_epochs = 1
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (x_i,y_i) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        x_i = x_i.reshape(-1, 50, 300).to(device)
        y_i = y_i.to(device)
        
        # Forward pass
        outputs = GRU_model(x_i)
        loss = criterion(outputs, y_i)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/1], Step [100/1600], Loss: 0.6452
Epoch [1/1], Step [200/1600], Loss: 0.5023
Epoch [1/1], Step [300/1600], Loss: 0.4549
Epoch [1/1], Step [400/1600], Loss: 0.3687
Epoch [1/1], Step [500/1600], Loss: 0.3416
Epoch [1/1], Step [600/1600], Loss: 0.3899
Epoch [1/1], Step [700/1600], Loss: 0.3783
Epoch [1/1], Step [800/1600], Loss: 0.4301
Epoch [1/1], Step [900/1600], Loss: 0.3203
Epoch [1/1], Step [1000/1600], Loss: 0.3960
Epoch [1/1], Step [1100/1600], Loss: 0.3546
Epoch [1/1], Step [1200/1600], Loss: 0.3367
Epoch [1/1], Step [1300/1600], Loss: 0.3432
Epoch [1/1], Step [1400/1600], Loss: 0.3284
Epoch [1/1], Step [1500/1600], Loss: 0.3906
Epoch [1/1], Step [1600/1600], Loss: 0.3222


In [62]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 50, 300).to(device)
        labels = labels.to(device)
        outputs = GRU_model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of binary GRU google model: {acc} %')

Accuracy of binary GRU google model: 87.3825 %


In [63]:
binary_GRU_opm = acc

# Ternary GRU my model

In [70]:
ternary_data = list(zip(new_test.review_body.values,new_test.y.values))

train_set, test_set = train_test_split(ternary_data, random_state = 19, test_size = 0.2)

test = data_vector(test_set, opm=False )

train = data_vector(train_set, opm=False)


# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
#valid_size = 0.2
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=True, num_workers=0)

test_loader=torch.utils.data.DataLoader(test, batch_size=batch_size, drop_last=True,num_workers=0)

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

GRU_model = GRU(300, 50, 2, 3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(GRU_model.parameters(), lr=0.002)  

# Train the model
num_epochs = 1
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (x_i,y_i) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        x_i = x_i.reshape(-1, 50, 300).to(device)
        y_i = y_i.to(device)
        
        # Forward pass
        outputs = GRU_model(x_i)
        loss = criterion(outputs, y_i)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/1], Step [100/2000], Loss: 0.9219
Epoch [1/1], Step [200/2000], Loss: 0.7806
Epoch [1/1], Step [300/2000], Loss: 0.6303
Epoch [1/1], Step [400/2000], Loss: 0.6929
Epoch [1/1], Step [500/2000], Loss: 0.7986
Epoch [1/1], Step [600/2000], Loss: 0.7141
Epoch [1/1], Step [700/2000], Loss: 0.6560
Epoch [1/1], Step [800/2000], Loss: 0.6516
Epoch [1/1], Step [900/2000], Loss: 0.6664
Epoch [1/1], Step [1000/2000], Loss: 0.7144
Epoch [1/1], Step [1100/2000], Loss: 0.7327
Epoch [1/1], Step [1200/2000], Loss: 0.6977
Epoch [1/1], Step [1300/2000], Loss: 0.7030
Epoch [1/1], Step [1400/2000], Loss: 0.6500
Epoch [1/1], Step [1500/2000], Loss: 0.6327
Epoch [1/1], Step [1600/2000], Loss: 0.6348
Epoch [1/1], Step [1700/2000], Loss: 0.7318
Epoch [1/1], Step [1800/2000], Loss: 0.7805
Epoch [1/1], Step [1900/2000], Loss: 0.6831
Epoch [1/1], Step [2000/2000], Loss: 0.6341


In [75]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 50, 300).to(device)
        labels = labels.to(device)
        outputs = GRU_model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of ternary GRU my model: {acc} %')

Accuracy of ternary GRU my model: 72.188 %


In [76]:
binary_GRU_my_model = acc

In [77]:
binary_GRU_my_model

72.188

# Ternary GRU google model

In [78]:
ternary_data = list(zip(new_test.review_body.values,new_test.y.values))

train_set, test_set = train_test_split(ternary_data, random_state = 19, test_size = 0.2)

test = data_vector(test_set, opm=True )

train = data_vector(train_set, opm=True)


# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
#valid_size = 0.2
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=True, num_workers=0)

test_loader=torch.utils.data.DataLoader(test, batch_size=batch_size, drop_last=True,num_workers=0)

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

GRU_model = GRU(300, 50, 2, 3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(GRU_model.parameters(), lr=0.002)  

# Train the model
num_epochs = 1
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (x_i,y_i) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        x_i = x_i.reshape(-1, 50, 300).to(device)
        y_i = y_i.to(device)
        
        # Forward pass
        outputs = GRU_model(x_i)
        loss = criterion(outputs, y_i)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/1], Step [100/2000], Loss: 0.9518
Epoch [1/1], Step [200/2000], Loss: 0.8481
Epoch [1/1], Step [300/2000], Loss: 0.7337
Epoch [1/1], Step [400/2000], Loss: 0.8770
Epoch [1/1], Step [500/2000], Loss: 0.7802
Epoch [1/1], Step [600/2000], Loss: 0.7466
Epoch [1/1], Step [700/2000], Loss: 0.8227
Epoch [1/1], Step [800/2000], Loss: 0.7063
Epoch [1/1], Step [900/2000], Loss: 0.7319
Epoch [1/1], Step [1000/2000], Loss: 0.6627
Epoch [1/1], Step [1100/2000], Loss: 0.6182
Epoch [1/1], Step [1200/2000], Loss: 0.7480
Epoch [1/1], Step [1300/2000], Loss: 0.6651
Epoch [1/1], Step [1400/2000], Loss: 0.6778
Epoch [1/1], Step [1500/2000], Loss: 0.6995
Epoch [1/1], Step [1600/2000], Loss: 0.6357
Epoch [1/1], Step [1700/2000], Loss: 0.5874
Epoch [1/1], Step [1800/2000], Loss: 0.6396
Epoch [1/1], Step [1900/2000], Loss: 0.6808
Epoch [1/1], Step [2000/2000], Loss: 0.7171


In [79]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 50, 300).to(device)
        labels = labels.to(device)
        outputs = GRU_model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of ternary GRU google model: {acc} %')

Accuracy of ternary GRU google model: 71.422 %


## GRU conclusion

Accuracy of ternary GRU google model: 71.422 %

Accuracy of ternary GRU my model: 72.188 %

Accuracy of binary GRU google model: 87.3825 %

Accuracy of binary GRU my model: 87.6475 %